In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from PEPit import PEP
from PEPit.functions import SmoothStronglyConvexFunction

def alt_gda_pep(mu, L, n_steps):
    problem = PEP()

    # 定义强凸强凹函数f(x,y)
    func_x = problem.declare_function(SmoothStronglyConvexFunction, L=L, mu=mu)
    func_y = problem.declare_function(SmoothStronglyConvexFunction, L=L, mu=mu)

    # 定义起始点
    x0 = problem.set_initial_point()
    y0 = problem.set_initial_point()

    # 定义最优点 (Nash Equilibrium)
    xs = func_x.stationary_point()
    ys = func_y.stationary_point()
    # 初始误差
    problem.set_initial_condition((x0 - xs)**2 + (y0 - ys)**2 <= 1)

    # 交替梯度下降上升迭代 (Alt-GDA)
    x, y = x0, y0
    eta = 1 / (2 * L)  # 步长，根据理论设置
    for _ in range(n_steps):
        x = x - eta * func_x.gradient(x)
        y = y + eta * func_y.gradient(y)

    # 设置目标为最优点的距离
    problem.set_performance_metric((x - xs)**2 + (y - ys)**2)

    # 求解PEP问题
    pep_value = problem.solve(verbose=0)

    return pep_value

if __name__ == "__main__":
    kappas = np.logspace(1, 3)  # 条件数区间
    complexities = []
    n_steps = 10  # 固定迭代步数

    for kappa in kappas:
        mu = 0.1
        L = kappa * mu
        complexity = alt_gda_pep(mu, L, n_steps)
        complexities.append(complexity)

    plt.figure(figsize=(6, 4))
    plt.loglog(kappas, complexities, 'o-', label='Alt-GDA worst-case complexity (PEP)')
    plt.xlabel('Condition number κ')
    plt.ylabel('Worst-case complexity')
    plt.grid(True, which='both', ls='--', alpha=0.4)
    plt.legend()
    plt.title('Alt-GDA complexity via PEP')
    plt.tight_layout()
    plt.show()

    # 用线性回归估计log-log斜率，验证复杂度指数
    slope, intercept = np.polyfit(np.log(kappas), np.log(complexities), 1)
    print(f"估计复杂度为O(kappa^{slope:.3f})")


In [2]:
from PEPit import PEP
from PEPit.operators import LipschitzStronglyMonotoneOperator

# 设置问题参数
mu_x, mu_y = 0.1, 0.1
L_x, L_y = 1, 1
L_xy = 1

# Lipschitz 常数估计 (可更精细)
L = max(L_x, L_y, L_xy)

# 初始化PEP问题
problem = PEP()

# 声明Lipschitz单调算子
F = problem.declare_function(LipschitzStronglyMonotoneOperator, L=L, mu = mu_x)

# 初始化点 z0 = (x0, y0)
z0 = problem.set_initial_point()

# 设置步长 (来自论文推荐)
alpha = 1 / (2 * L_x)
beta = 1 / (2 * L_y)

# Alt-GDA更新规则：
x0, y0 = z0.decompose()
grad_x0, grad_y0 = F.gradient(z0).decompose()

# Alt-GDA第一步 (x 更新)
x1 = x0 - alpha * grad_x0

# Alt-GDA第二步 (y 更新，注意这里用了 x1 而非 x0)
z_half = problem.set_point((x1, y0))
_, grad_y_half = F.gradient(z_half).decompose()
y1 = y0 - beta * grad_y_half

# 组合回 z1
z1 = problem.set_point((x1, y1))

# 设置收敛性能度量为算子范数的平方
problem.set_performance_metric(F.gradient(z1)**2)

# 求解PEP问题
pep_value = problem.solve(verbose=True)

print(f'Worst-case performance (operator norm squared): {pep_value:.6f}')

AttributeError: 'Point' object has no attribute 'decompose'

In [ ]:
from PEPit import PEP
from PEPit.operators import LipschitzStronglyMonotoneOperator

# 设置问题参数
mu_x, mu_y = 0.1, 0.1
L_x, L_y = 1, 1
L_xy = 1
L = max(L_x, L_y, L_xy)

# 初始化PEP问题
problem = PEP()

# 声明Lipschitz单调算子
F = problem.declare_function(LipschitzStronglyMonotoneOperator, L=L, mu = mu_x)

# 初始化点(x0, y0)
x0 = problem.set_initial_point()
y0 = problem.set_initial_point()
z0 = (x0, y0)

# 设置步长 (Alt-GDA 推荐值)
alpha = 1 / (2 * L_x)
beta = 1 / (2 * L_y)

# Alt-GDA 更新
grad_x0, _ = F.gradient(z0)
x1 = x0 - alpha * grad_x0

z_half = problem.set_point((x1, y0))
_, grad_y_half = F.gradient(z_half)
y1 = y0 - beta * grad_y_half

# 更新后的新点
z1 = problem.set_point((x1, y1))

# 设置性能度量 (最常用的是算子范数平方)
problem.set_performance_metric(F.gradient(z1)**2)

# 求解PEP问题
pep_value = problem.solve(verbose=True)

print(f'Worst-case performance (operator norm squared): {pep_value:.6f}')

AssertionError: 